In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define paths

In [2]:
train_dir = 'split_data/train'
val_dir = 'split_data/val'
test_dir = 'split_data/test'

# Parameters

In [3]:
img_height, img_width = 128, 128
batch_size = 64
seed = 42

# Create ImageDataGenerator instances

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed
)

Found 1862 images belonging to 9 classes.


# Validation data generator

In [6]:
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed
)

Found 399 images belonging to 9 classes.


# Test data generator

In [7]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed
)

Found 399 images belonging to 9 classes.


### Base model (InceptionV3)

train from scratch


In [8]:
base_model = tf.keras.applications.InceptionV3(include_top=False,
    weights=None,
    pooling='avg',input_shape = (128,128,3))

In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

In [11]:
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())
output =  tf.keras.layers.Dense(len(class_names), activation='softmax')(model.output)
new_model = tf.keras.Model(model.input, output)

# Define early stopping

In [12]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    verbose=1,
    patience=2,
    mode='max',
    restore_best_weights=True
)

# Compile the model

In [13]:
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False,
    label_smoothing=0,
    reduction="auto",
    name="categorical_crossentropy"), metrics=['accuracy'])

# Train the model

In [14]:
history = new_model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[es_callback]
)


Epoch 1/10
30/30 [==============================] - 113s 3s/step - loss: 2.2089 - accuracy: 0.1090 - val_loss: 2.2043 - val_accuracy: 0.1103
Epoch 2/10
30/30 [==============================] - 102s 3s/step - loss: 2.2031 - accuracy: 0.1165 - val_loss: 2.2008 - val_accuracy: 0.1128
Epoch 3/10
30/30 [==============================] - 88s 3s/step - loss: 2.2026 - accuracy: 0.1101 - val_loss: 2.1995 - val_accuracy: 0.1128
Epoch 4/10
30/30 [==============================] - 89s 3s/step - loss: 2.1996 - accuracy: 0.1122 - val_loss: 2.1983 - val_accuracy: 0.1128
Epoch 4: early stopping


# Evaluate the model on the test set

In [15]:
test_loss, test_accuracy = new_model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

7/7 [==============================] - 14s 2s/step - loss: 2.2008 - accuracy: 0.1128
Test Loss: 2.2008092403411865, Test Accuracy: 0.11278195679187775


In [16]:
image = 'prediction/g.jpeg'
image = tf.keras.utils.load_img(image, target_size=(img_height,img_width))
img_arr = tf.keras.utils.array_to_img(image)
img_bat=tf.expand_dims(img_arr,0)

In [17]:
predict = new_model.predict(img_bat)

1/1 [==============================] - 5s 5s/step


In [18]:
score = tf.nn.softmax(predict)

In [19]:
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

In [20]:
class_names

['Duryodhan',
 'Ganga',
 'Lali',
 'Parashuram',
 'Raju',
 'Sivilbahadur',
 'Sonu',
 'Tirathram',
 'Yoglaxmi']

In [21]:
print('Elephant in image is {} with accuracy of {:0.2f}'.format(class_names[np.argmax(score)],np.max(score)*100))

Elephant in image is Raju with accuracy of 11.64


In [22]:
new_model.save('InceptionV3_elephant_model.h5')